In [20]:
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt

In [21]:
class Config:
    ROOT_DIR = './kaggle/input/asl-signs'
    TRAIN_DATA = './kaggle/input/asl-signs/train.csv'
    EXTENDED_TRAIN_DATA = './kaggle/input/gislr-extended-train-dataframe/extended_train.csv'
    TRAIN_DIR = './kaggle/input/asl-signs/train_landmark_files'
    PREDICTION_INDEX_MAP = './kaggle/input/asl-signs/sign_to_prediction_index_map.json'
    

In [28]:
train_df = pd.read_csv(Config.EXTENDED_TRAIN_DATA)
train_df.path = train_df.path.apply(lambda path: f".{path}")

In [29]:
train_df.head()

,path,participant_id,sequence_id,sign,start_frame,end_frame,total_frames,face_count,face_nan_count,pose_count,...,left_hand_count,left_hand_nan_count,right_hand_count,right_hand_nan_count,x_min,x_max,y_min,y_max,z_min,z_max
0,./kaggle/input/asl-signs/train_landmark_files/...,26734,1000035562,blow,20,42,23,10764,0,759,...,483,483,483,252,-0.031811,1.294350,-2.147826,-0.224151,-2.782624,1.910815
1,./kaggle/input/asl-signs/train_landmark_files/...,28656,1000106739,wait,29,39,11,5148,0,363,...,231,231,231,189,-0.017062,1.015931,-2.222732,-0.329273,-2.543970,1.627621
2,./kaggle/input/asl-signs/train_landmark_files/...,16069,100015657,cloud,103,207,105,49140,0,3465,...,2205,1617,2205,2205,-0.042923,1.197836,-2.591290,-0.248094,-2.838325,1.587503
3,./kaggle/input/asl-signs/train_landmark_files/...,25571,1000210073,bird,17,28,12,5616,0,396,...,252,252,252,0,-0.129268,1.156573,-2.294936,-0.310272,-3.018237,2.196296
4,./kaggle/input/asl-signs/train_landmark_files/...,62590,1000240708,owie,22,39,18,8424,0,594,...,378,378,378,0,0.072425,1.165405,-2.264609,-0.405282,-2.970331,1.360011


In [30]:
train_df.describe()

,participant_id,sequence_id,start_frame,end_frame,total_frames,face_count,face_nan_count,pose_count,pose_nan_count,left_hand_count,left_hand_nan_count,right_hand_count,right_hand_nan_count,x_min,x_max,y_min,y_max,z_min,z_max
count,94477.000000,9.447700e+04,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.0,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000,94477.000000
mean,33678.632366,2.149377e+09,30.421044,67.361379,37.935021,17753.589932,120.481133,1251.855700,0.0,796.635446,554.765911,796.635446,550.659134,-0.076621,1.101067,-2.352547,-0.256284,-2.769590,1.813202
std,16138.124387,1.239239e+09,29.630199,58.981009,44.177069,20674.868298,1048.131797,1457.843277,0.0,927.718449,781.974565,927.718449,816.502712,0.124995,0.138647,0.217907,0.086027,0.585033,0.600891
min,2044.000000,8.528200e+04,0.000000,1.000000,2.000000,936.000000,0.000000,66.000000,0.0,42.000000,0.000000,42.000000,0.000000,-1.334372,0.271741,-3.572496,-0.547721,-6.234318,0.113979
25%,25571.000000,1.078076e+09,16.000000,33.000000,12.000000,5616.000000,0.000000,396.000000,0.0,252.000000,126.000000,252.000000,63.000000,-0.152122,1.002237,-2.488619,-0.317737,-3.173018,1.396756
50%,32319.000000,2.154240e+09,21.000000,43.000000,22.000000,10296.000000,0.000000,726.000000,0.0,462.000000,294.000000,462.000000,273.000000,-0.064046,1.093523,-2.351512,-0.256689,-2.779032,1.794568
75%,49445.000000,3.218820e+09,29.000000,62.000000,44.000000,20592.000000,0.000000,1452.000000,0.0,924.000000,630.000000,924.000000,630.000000,0.011356,1.184765,-2.203060,-0.198166,-2.390240,2.209615
max,62590.000000,4.294915e+09,484.000000,546.000000,537.000000,251316.000000,58968.000000,17721.000000,0.0,11277.000000,10500.000000,11277.000000,11235.000000,0.532557,2.920505,-0.497782,2.301204,-0.310294,4.796592


In [31]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94477 entries, 0 to 94476
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   path                  94477 non-null  object 
 1   participant_id        94477 non-null  int64  
 2   sequence_id           94477 non-null  int64  
 3   sign                  94477 non-null  object 
 4   start_frame           94477 non-null  int64  
 5   end_frame             94477 non-null  int64  
 6   total_frames          94477 non-null  int64  
 7   face_count            94477 non-null  int64  
 8   face_nan_count        94477 non-null  int64  
 9   pose_count            94477 non-null  int64  
 10  pose_nan_count        94477 non-null  int64  
 11  left_hand_count       94477 non-null  int64  
 12  left_hand_nan_count   94477 non-null  int64  
 13  right_hand_count      94477 non-null  int64  
 14  right_hand_nan_count  94477 non-null  int64  
 15  x_min              

In [32]:
train_df.isnull().sum()

path                    0
participant_id          0
sequence_id             0
sign                    0
start_frame             0
end_frame               0
total_frames            0
face_count              0
face_nan_count          0
pose_count              0
pose_nan_count          0
left_hand_count         0
left_hand_nan_count     0
right_hand_count        0
right_hand_nan_count    0
x_min                   0
x_max                   0
y_min                   0
y_max                   0
z_min                   0
z_max                   0
dtype: int64

In [33]:
landmark_path_0 = pd.read_parquet(train_df['path'][0])
landmark_path_0.head()

,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152


In [34]:
landmark_path_0.shape

(12489, 7)

In [35]:
meta_data_df = train_df.groupby('sign').agg({'sequence_id': 'count', 'total_frames': 'sum'})
meta_data_df.columns = ['num_seq', 'num_frames']
meta_data_df['avg_frames'] = np.round(meta_data_df['num_frames'] / meta_data_df['num_seq'])

meta_data_df.head()

,num_seq,num_frames,avg_frames
sign,,,
TV,385,14197,37.0
after,347,10546,30.0
airplane,393,17274,44.0
all,386,14090,37.0
alligator,390,18021,46.0


In [36]:
print(f"Mean of the average frames: {meta_data_df['avg_frames'].mean()}")

Mean of the average frames: 37.852
